# Роль LayerNorm в nanoGPT: Эксперимент с градиентами

В этом блокноте мы возьмем архитектуру nanoGPT и сравним две версии:
1. **Stable**: Оригинальная архитектура с LayerNorm.
2. **Broken**: Та же архитектура, но с 'вырезанными' слоями LayerNorm.

Мы увидим, как быстро разваливается backprop без нормализации.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt

n_embd = 384
n_head = 6
n_layer = 24 # Увеличим глубину
block_size = 64
vocab_size = 65
device = 'cpu'
torch.manual_seed(1337)

## 1. Определение архитектуры с переключателем LayerNorm

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, use_ln=True):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.use_ln = use_ln
        if use_ln:
            self.ln1 = nn.LayerNorm(n_embd)
            self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        if self.use_ln:
            x = x + self.sa(self.ln1(x))
            x = x + self.ffwd(self.ln2(x))
        else:
            # БЕЗ LayerNorm
            x = x + self.sa(x)
            x = x + self.ffwd(x)
        return x

class GPT(nn.Module):
    def __init__(self, use_ln=True):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, use_ln=use_ln) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

## 2. Запуск сравнения: Stable vs Broken

In [ ]:
model_stable = GPT(use_ln=True)
model_broken = GPT(use_ln=False)
model_broken.load_state_dict(model_stable.state_dict(), strict=False)

xb = torch.randint(0, vocab_size, (1, block_size))
yb = torch.randint(0, vocab_size, (1, block_size))

def analyze(model):
    # 1. Forward pass с записью норм активаций
    activations_norms = []
    x = model.token_embedding_table(xb) + model.position_embedding_table(torch.arange(block_size))
    for block in model.blocks:
        x = block(x)
        activations_norms.append(x.norm().item())
    
    # 2. Backward pass для градиентов
    logits = model.lm_head(x)
    loss = F.cross_entropy(logits.view(-1, vocab_size), yb.view(-1))
    model.zero_grad()
    loss.backward()
    
    grads = [block.ffwd.net[0].weight.grad.norm().item() for block in model.blocks]
    return activations_norms, grads

norms_s, grads_s = analyze(model_stable)
norms_b, grads_b = analyze(model_broken)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# График Активаций (Forward)
ax1.plot(norms_s, 'g-o', label='With LayerNorm')
ax1.plot(norms_b, 'r-o', label='Without LayerNorm')
ax1.set_title('Рост Активаций (Forward Pass)')
ax1.set_xlabel('Block index')
ax1.set_ylabel('L2 Norm of Activations')
ax1.legend()
ax1.grid(True)

# График Градиентов (Backward)
ax2.plot(grads_s, 'g-o', label='With LayerNorm')
ax2.plot(grads_b, 'r-o', label='Without LayerNorm')
ax2.set_title('Масштаб Градиентов (Backward Pass)')
ax2.set_xlabel('Block index')
ax2.set_ylabel('Gradient Norm')
ax2.set_yscale('log')
ax2.legend()
ax2.grid(True)

plt.show()

print(f'Финальная норма активаций Broken: {norms_b[-1]:.2f}')
print(f'Финальная норма активаций Stable: {norms_s[-1]:.2f}')
print(f"\nБез LayerNorm активации растут в {norms_b[-1]/norms_s[-1]:.1f} раз сильнее!")

## Математическая природа взрыва градиентов

Взрыв градиентов — это следствие **цепного правила (Chain Rule)** при глубокой вложенности функций (слоев).

### 1. Формула Backpropagation
Представим сеть как цепочку преобразований: $y = f_L(f_{L-1}(\dots f_1(x) \dots))$.
Градиент функции потерь $\mathcal{L}$ по весам первого слоя $W_1$ вычисляется как произведение:

$$\frac{\partial \mathcal{L}}{\partial W_1} = \frac{\partial \mathcal{L}}{\partial y} \cdot \frac{\partial f_L}{\partial f_{L-1}} \cdot \frac{\partial f_{L-1}}{\partial f_{L-2}} \dots \frac{\partial f_1}{\partial W_1}$$

### 2. Матричное умножение
Для линейного слоя без активации производная перехода $\frac{\partial f_i}{\partial f_{i-1}}$ — это просто матрица весов $W_i^T$. 
Если у нас $L$ слоев, то градиент пропорционален произведению этих матриц:

$$\text{Gradient} \propto \prod_{i=2}^{L} W_i^T$$

Если сингулярные числа этих матриц $\sigma > 1$, то норма градиента растет экспоненциально:
$$\|\text{Gradient}\| \sim \sigma^L$$

### 3. Как это лечат современные архитектуры?

#### **LayerNorm**
Нормализует активации $x$, удерживая их дисперсию в разумных пределах. В обратном проходе LayerNorm работает как адаптивный делитель, уменьшающий слишком большие градиенты.

#### **Residual Connections (Остаточные связи)**
Вместо прохода сквозь веса, градиент может использовать "магистраль": $x_{next} = x + f(x)$.
Производная такого узла:
$$\frac{\partial (x + f(x))}{\partial x} = 1 + f'(x)$$
Единица здесь гарантирует, что градиент не будет умножаться на веса, если они слишком велики или малы, обеспечивая стабильный поток информации.

In [ ]:
# Визуализация градиентов в ЛИНЕЙНОЙ шкале
plt.figure(figsize=(10, 5))
plt.plot(grads_s, 'g-o', label='With LayerNorm (Stable)')
plt.plot(grads_b, 'r-o', label='Without LayerNorm (Broken)')
plt.title('Масштаб Градиентов (Линейная шкала)')
plt.xlabel('Block index')
plt.ylabel('Gradient Norm')
plt.legend()
plt.grid(True)
plt.show()

print(f'Средний градиент Stable: {sum(grads_s)/len(grads_s):.4f}')
print(f'Средний градиент Broken: {sum(grads_b)/len(grads_b):.4f}')